In [ ]:
Group Project Proposal

In [ ]:
Introduction (Alvin):




In [ ]:
Preliminary exploratory data analysis (Wayne):




In [ ]:
Methods (Peter):




In [ ]:
Expected outcomes and significance (Peter):



